### Feed Forward

In [1]:
%run core.ipynb

In [157]:
def feedforward(a, N, x):
    for w, b in N:
        x = a(np.dot(w, x) + b)
    return x


In [158]:
a = lambda x: np.multiply(x, 1)
N = [
    (np.matrix((2, 2), 
               (1, -1),
               (2, 2)),  
     np.vector(1, -1, 0)),
    (np.matrix((1, 2, 2),
               (0, 0, 3),
               (1, 2, 2)), 
     np.vector(1, -1, -2)),
    (np.matrix((2, 2, 1),), 
     np.vector(1,)),
]
x = np.vector(1, -1)

feedforward(a, N, x)
     

array([8])

In [159]:
def get_random_weights_and_biases(k, l, m=None, layers=0):
    N = [(np.random_matrix(l, k), np.random_vector(l))]
    for _ in range(layers):
        N.append((np.random_matrix(l, l), np.random_vector(l)))
    if m is not None: 
        N.append((np.random_matrix(m, l), np.random_vector(m)))
    return N


In [160]:
a = np.sigmoid
N = get_random_weights_and_biases(2, 3, 2, layers=3)
x = np.random_vector(2)
feedforward(a, N, x)


array([0.88884483, 0.90324136])

In [161]:
class FeedForward:
    
    def __init__(self, dim, layers, a):
        self.net = get_random_weights_and_biases(*dim, layers)
        self.a = a

    def __call__(self, x):
        return self.feedforward(x)
    
    def feedforward(self, x):
        return feedforward(self.a, self.net, x)


In [162]:
f = FeedForward(dim=(2, 3, 2), layers=3, a=np.sigmoid)

x = np.random_vector(2)
f(x)


array([0.91842484, 0.89931237])

In [163]:
f = FeedForwardSingleLayer(dim=(3, 2), a=np.sigmoid, da=np.dx_sigmoid)
f.net

[(array([[0.22638905, 0.60920239, 0.31226211],
         [0.63902029, 0.65569293, 0.72866763]]),
  array([0.89337596, 0.82689486]))]

In [164]:
class FeedForwardSingleLayer(FeedForward):

    def __init__(self, dim, a, da):
        super().__init__(dim=(*dim, None), layers=0, a=a)
        self.da = da

    def train(self, T, s=0.01, epochs=50):
        for _ in range(epochs):
            for x, y in T:
                z = feedforward(np.linear, self.net, x)
                az, daz = self.a(z), norm(self.da(z))
                d = norm(y - az)
                for w, b in self.net:
                    w += s * d * daz * x
                    b += s * d * daz


In [165]:
from dec2bin import int2dec, int2bin, bin2int


class Dec2Bin(FeedForwardSingleLayer):

    def __init__(self, a=np.sigmoid, da=np.dx_sigmoid, s=0.01, epochs=100):
        super().__init__(dim=(10, 4), a=a, da=da)
        self.train(self.get_test_data(), s, epochs)

    def __call__(self, n):
        return super().__call__(int2dec(n))
    
    def get_test_data(self):
        D = {n: int2dec(n) for n in range(10)}
        B = {n: int2bin(n) for n in range(10)}
        T = []
        for n in range(10):
            T. append((np.vector(*D[n]), np.vector(*B[n])))
        return T

In [174]:


s, epochs = 0.01, 750
dec2bin = Dec2Bin()
dec2bin.train(get_dec2bin_test_data(), s, epochs)
print(dec2bin.net)


[(array([[1.12411585, 0.44125335, 0.7149309 , 1.28647672, 0.50269358,
        1.18478086, 0.76964197, 0.35260409, 0.4243595 , 1.04749984],
       [0.92795537, 1.32128869, 0.99333794, 0.63190382, 0.93647869,
        0.68583767, 1.12269588, 1.0749699 , 1.31327723, 0.57436938],
       [0.84759195, 1.24556085, 0.86864931, 1.19908472, 0.80400178,
        0.72431964, 1.18388543, 1.14376884, 1.3802643 , 0.90669437],
       [0.84928862, 1.1197685 , 0.6302814 , 0.97702678, 0.90983906,
        1.1074282 , 0.97532494, 0.48799452, 1.16964972, 1.02512125]]), array([4.75028457, 4.68855662, 4.388732  , 4.51625203]))]


In [176]:
classify = lambda x: heaviside(x, offset=0.5)


def check_dec2bin_delta_rule():
    for n in range(10):
        t = dec2bin(n)
        print(t, bin2int(t))
        continue
        X = tuple(int2bin(n))
        Y = tuple(a(x) for x in np.dot(W, int2dec(n)) + B)
        Z = tuple(classify(t) for t in Y)
        if X == Z:
            print(f"{n} =	{_rnd(Y, n=2)} =	{Z}")
        else:
            print(_red(f"{n} ≠	{_rnd(Y, n=2)} =	{Z}	≠	{X}"))

            
check_dec2bin_delta_rule()

[0.99734963 0.99664444 0.99514273 0.99568388] 15
[0.9948377  0.99771532 0.99670622 0.99668555] 15
[0.99604732 0.99685204 0.99524147 0.99465612] 15
[0.99773862 0.99552001 0.99655326 0.9961898 ] 15
[0.99513789 0.99667225 0.99493182 0.99593153] 15
[0.99750222 0.99574967 0.99452244 0.99664535] 15
[0.99625289 0.99722581 0.99650171 0.99618346] 15
[0.99437179 0.99709336 0.99636195 0.99386142] 15
[0.99475197 0.99769735 0.99711238 0.99684317] 15
[0.99714356 0.99526138 0.99541482 0.99636458] 15
